In [ ]:
# for domonkos to run on COLAB
"""
import os
os.chdir('/content/drive/MyDrive/python/Computer_Vision/TDT4265_Project/')

from google.colab import drive
drive.mount('/content/drive')
"""

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from src.data import load_train_test_val
from src.visualize import plot_loss_acc
from src.model import Unet2D
from src.train import train
from src.utils import save_result
from src.test import test
from src.metrics import acc_metric

from params import *


import torch
from torch import nn

## Prepare data

In [ ]:
train_data, test_data, valid_data = load_train_test_val(DATA_PARAMS, PREP_STEPS, TRAIN_TRANSFORMS)

## Train

In [ ]:
# MODEL: Unet2D (one input channel, 4 output channels)
# Outputs: Probabilities for each class for each pixel in different layer)
unet = Unet2D(1, out_channels=4)

loss_fn = nn.CrossEntropyLoss()
opt = torch.optim.Adam(unet.parameters(), lr=LEARN_RATE)

In [ ]:
if LOAD:
    unet.load_state_dict(torch.load(MODEL_PATH+FILE_NAME))

In [ ]:
from datetime import datetime

start = datetime.now()
# Training process
train_loss, test_loss, train_acc, test_acc = train(unet, train_data, test_data, loss_fn, opt, acc_metric, epochs=EPOCHS)

end = datetime.now()

print("Elapsed time is {}".format(str(end-start)))

In [ ]:
plot_loss_acc(train_loss, test_loss, train_acc, test_acc)

## Test

In [ ]:
accuracy, average_dice, class_dice = test(unet, valid_data, True)

In [ ]:
# save the result
msg = 'Test the saving function'

file_name = 'test1'

# Save the model
save_result(unet, file_name, accuracy, average_dice, class_dice, msg = msg )